In [86]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_palette("Set2")
import warnings
warnings.filterwarnings("ignore")
import datetime as dt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import scipy.stats as stats
#pd.set_option('display.max_rows', None)

In [110]:
df = pd.read_csv("C:/Users/Ekaterina.Koobas/Desktop/volga8.csv", sep=";")

In [111]:
df.index = df.index + 1

In [112]:
display(df.head())

,Месяц,День,Территория,Регион,ТТ,"Потери, кг\шт","Выручка, кг\шт"
1,сент. 2024,30.09.2024,Центр,Регион МО,"006 - г.Апрелевка, Больничная",0.0,3.0
2,сент. 2024,30.09.2024,Центр,Регион МО,"007 - г.Раменское, Гурьева ул.",1.0,0.0
3,сент. 2024,30.09.2024,Центр,Регион МО,"008 - г.Люберцы, Егорьевское ш",1.0,1.0
4,сент. 2024,30.09.2024,Центр,Регион МО,"015 - г.Дмитров, Загорская ул.",3.0,0.0
5,сент. 2024,30.09.2024,Центр,Регион МО,"021 - г.Серпухов, Ворошилова у",0.0,1.0


In [113]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414417 entries, 1 to 414417
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Месяц           119028 non-null  object 
 1   День            119028 non-null  object 
 2   Территория      119028 non-null  object 
 3   Регион          119028 non-null  object 
 4   ТТ              119028 non-null  object 
 5   Потери, кг\шт   119028 non-null  float64
 6   Выручка, кг\шт  119028 non-null  float64
dtypes: float64(2), object(5)
memory usage: 22.1+ MB
None


In [114]:
df['Off-take'] = df['Потери, кг\шт'] + df['Выручка, кг\шт']

In [115]:
display(df.head())

,Месяц,День,Территория,Регион,ТТ,"Потери, кг\шт","Выручка, кг\шт",Off-take
1,сент. 2024,30.09.2024,Центр,Регион МО,"006 - г.Апрелевка, Больничная",0.0,3.0,3.0
2,сент. 2024,30.09.2024,Центр,Регион МО,"007 - г.Раменское, Гурьева ул.",1.0,0.0,1.0
3,сент. 2024,30.09.2024,Центр,Регион МО,"008 - г.Люберцы, Егорьевское ш",1.0,1.0,2.0
4,сент. 2024,30.09.2024,Центр,Регион МО,"015 - г.Дмитров, Загорская ул.",3.0,0.0,3.0
5,сент. 2024,30.09.2024,Центр,Регион МО,"021 - г.Серпухов, Ворошилова у",0.0,1.0,1.0


In [116]:
print("Количество дубликатов:", df.duplicated().sum())

Количество дубликатов: 295388


In [117]:
display(df['Потери, кг\шт'].sum())
display(df['Выручка, кг\шт'].sum())
display(df['Off-take'].sum())

54027.0

122267.0

176294.0

In [118]:
48456 + 458419

506875

In [119]:
df_svod1 = pd.pivot_table(
    df, 
    index=["Территория", "Регион"],
    columns=["Месяц"],
    values=["День", "ТТ", "Off-take"],
    aggfunc={
        "День": lambda x: len(np.unique(x)),
        "ТТ": lambda x: len(np.unique(x)),
        "Off-take": np.sum
    }, 
    fill_value=0
)

# Упорядочиваем столбцы так, чтобы каждая плановая дата поставки содержала данные "Заказано", "Поставлено", "SL, %"
df_svod1 = df_svod1.sort_index(axis=1, level=[1, 0])

# Перемещаем столбец "Off-take" в конец
columns_order = [col for col in df_svod.columns if col[0] != 'Off-take'] + [col for col in df_svod.columns if col[0] == 'Off-take']
df_svod1 = df_svod1[columns_order]

KeyError: "None of [MultiIndex([(    'День', 'сен.24'),\n            (      'ТТ', 'сен.24'),\n            ('Off-take', 'сен.24')],\n           names=[None, 'Месяц'])] are in the [columns]"

In [120]:
display(df_svod1)

Off-take      День        ТТ  \
Месяц                                   авг. 2024 авг. 2024 авг. 2024   
Территория Регион                                                       
Волга      Регион Волжск                      4.0         2         1   
           Регион Йошкар-Ола                 23.0         8         7   
           Регион Казань                    355.0        12        80   
           Регион Нижний Новгород           402.0        12       101   
           Регион Саранск                    31.0         9        10   
           Регион Саров                      33.0        12         6   
           Регион Чебоксары                  42.0        11        14   
Центр      Регион Владимир                 3621.0        31       298   
           Регион Калуга                   2241.0        31       195   
           Регион МО                      22126.0        31      1653   
           Регион Москва                  22648.0        31      1491   
           Регион Рязанское направление    2413.0        31       181   
           Регион Тверское направление     1185.0        31        93   
           Регион Тверь                    2265.0        31       169   
           Регион Тверь область            1221.0        31       100   
Юг         Регион Астрахань                 169.0        25        20   
           Регион Балаково                   29.0         9         2   
           Регион Волгоград                 229.0        21        47   
           Регион Новочеркасск               50.0        18         4   
           Регион Пенза                     204.0        30        16   
           Регион Ростов-на-Дону            246.0        31        12   
           Регион Самара                   1455.0        31       100   
           Регион Саратов                   333.0        31        27   
           Регион Сызрань                   111.0        26         9   
           Регион Тольятти                  229.0        31        17   
           Регион Ульяновск                 260.0        29        19   

                                         Off-take      День        ТТ  \
Месяц                                   июл. 2024 июл. 2024 июл. 2024   
Территория Регион                                                       
Волга      Регион Волжск                      0.0         0         0   
           Регион Йошкар-Ола                  0.0         0         0   
           Регион Казань                      0.0         0         0   
           Регион Нижний Новгород             0.0         0         0   
           Регион Саранск                     0.0         0         0   
           Регион Саров                       0.0         0         0   
           Регион Чебоксары                   0.0         0         0   
Центр      Регион Владимир                 1655.0        31       206   
           Регион Калуга                    857.0        28       145   
           Регион МО                      15824.0        31      1477   
           Регион Москва                  18351.0        31      1404   
           Регион Рязанское направление    1215.0        31       135   
           Регион Тверское направление      686.0        31        74   
           Регион Тверь                    1307.0        31       155   
           Регион Тверь область             641.0        31        77   
Юг         Регион Астрахань                 560.0        31        36   
           Регион Балаково                   31.0        15         2   
           Регион Волгоград                1722.0        31       106   
           Регион Новочеркасск               18.0         6         1   
           Регион Пенза                     188.0        30        14   
           Регион Ростов-на-Дону            209.0        30        12   
           Регион Самара                   1600.0        31       102   
           Регион Саратов                   357.0        31        25   
           Регион Сызрань

In [121]:
df_svod1.to_excel(r"C:/Users/Ekaterina.Koobas/Desktop/case_volga_8.xlsx")